# Hive assignment 1

The purpose of this task is to create an external table on the posts data of the `stackoverflow.com` website.

### Instructions

The purpose of this task is to create an external table on the posts data of the stackoverflow.com website.

Create your own database and 'use' it. Create external table 'posts_sample_external' over the sample dataset with posts in '/data/stackexchange1000' directory. Create managed table 'posts_sample' and populate with the data from the external table. 'Posts_sample' table should be partitioned by year and by month of post creation. Provide output of query which selects lines number per each partition in the format:
    
    year <tab> month <tab> lines count

where year in YYYY format and month in YYYY-MM format. The result is the 3th line of the last query output.

The result on the sample dataset:

    2008    2008-10 73





## Step 1. Intro. Creation of the DB

Let's create the sandbox database where you will complete your assignment.

<b>Note!</b> This code shouldn't be in your submission. Please, remove this code from the notebook before submission.

In [1]:
%%writefile creation_db.hql
DROP DATABASE IF EXISTS demodb CASCADE;

Writing creation_db.hql


In [2]:
%%writefile -a creation_db.hql
CREATE DATABASE demodb LOCATION '/user/jovyan/test_metastore';

Appending to creation_db.hql


In [3]:
! hive -f creation_db.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.09 seconds
OK
Time taken: 0.559 seconds


**Don't forget to remove this code before submission!**


## Step 2. Exploration of the dataset

Okay, we have created the database. Let's create your own table for users and posts.

First of all, let's watch at the datasets for `users` which are located at `/data/stackexchange1000/posts` and for `posts` which is located at `/data/stackexchange1000/users`. Print the first three rows of those datasets.


In [4]:
# Your code here. Print the first three rows of posts
!cat /data/stackexchange1000/posts/part-00000 | head -n 3

reporter:status:Reading 	
<row Id="1394" PostTypeId="2" ParentId="1390" CreationDate="2008-08-04T16:38:03.667" Score="16" Body="&lt;p&gt;Not sure how credible &lt;a href=&quot;http://www.builderau.com.au/program/windows/soa/Getting-started-with-Windows-Server-2008-Core-edition/0,339024644,339288700,00.htm&quot; rel=&quot;nofollow noreferrer&quot;&gt;this source is&lt;/a&gt;, but:&lt;/p&gt;&#xA;&#xA;&lt;blockquote&gt;&#xA;  &lt;p&gt;The Windows Server 2008 Core edition can:&lt;/p&gt;&#xA;  &#xA;  &lt;ul&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run the file server role.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run the Hyper-V virtualization server role.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run the Directory Services role.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run the DHCP server role.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run the IIS Web server role.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run the DNS server role.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run 

In [5]:
# Your code here. Print the first three rows of users
!cat /data/stackexchange1000/users/part-00000 | head -n 3

reporter:status:Reading 	
<row Id="756" Reputation="2358" CreationDate="2008-08-08T15:31:50.013" DisplayName="Simon Gillbee" LastAccessDate="2016-12-09T15:38:03.453" WebsiteUrl="http://simon.gillbee.com" Location="Pearland, TX" AboutMe="&lt;p&gt;Personally, I am a husband, step-father, grandfather, Christ-worshiper, singer, worship leader, computer programmer, reader, game player, kite flyer, generally all around fun guy (or that fungi?).&lt;/p&gt;&#xA;&#xA;&lt;p&gt;Professionally, I've been developing both commercial and proprietary systems for the last 20 years. These days I'm primarily writing enterprise-scale client and server software using .NET and loving it!&lt;/p&gt;&#xA;&#xA;&lt;h1&gt;SOreadytohelp&lt;/h1&gt;&#xA;" Views="478" UpVotes="352" DownVotes="25" Age="45" AccountId="587" />	
<row Id="2050" Reputation="4177" CreationDate="2008-08-20T00:32:49.217" DisplayName="Eric Platon" LastAccessDate="2016-12-10T22:24:27.217" WebsiteUrl="" Location="Tokyo, Japan" AboutMe="" Views="3

As you can see, those rows contain some information about posts and users in XML format.

<b>Question.</b> Which fields for users and posts do you think are the most important for the analysis? And for joining tables? 

<h3><b>Please, check your answer with this information!</b></h3>

So, the lines not started with the "row" tags should be ignored. The valid row contains the following fields and their order is not defined:

* Id (integer) - id of the post
* PostTypeId (integer: 1 or 2) - 1 for questions, 2 for answers
* CreationDate (date) - post creation date in the format "YYYY-MM-DDTHH:MM:SS.ms"
* Tags (string, optional) - list of post tags, each tag is wrapped with html entities `&lt;` and `&gt;`
* OwnerUserId (integer, optional) - user id of the post's author
* ParentId (integer, optional) - for answers - id of the question
* Score (integer) - score (votes) of a question or an answer, can be negative (!)
* FavoriteCount (integer, optional) - how many times the question was added in the favorites

The second part of the dataset contains StackOverflow users.

The fields are the following and their order is also not defined:

* Id (integer) - user id
* Reputation (integer) - user's reputation
* CreationDate (string) - creation date in the format "YYYY-MM-DDTHH:MM:SS.ms"
* DisplayName (string) - user's name
* Location (string, options) - user's country
* Age (integer, optional) - user's age

## Step 3. Train your regexp skills

In this step you will find out how to parse information for complex rows! You will try to create some examples for parsing! There are some general rules for parsing:

1. To create a regular expression, which describes strings containing two patterns, where the order of the patterns is not defined use the following so-called ‘positive lookahead assertion’ with `?=` group modifier. For example, both strings “Washington Irving” and “Irving Washington” match the pattern:
```
(?=.*Washington)(?=.*Irving)
```.
2. To capture groups use round brackets. So, the pattern: `(?=.*(Washington))(?=.*(Irving))` captures `Washington` and `Irving` from both strings: "William Arthur Irving Washington was an English first-class Cricketer" and: “Washington Irving was an American writer”.
3. Use `\b` to specify boundaries of words and increase accuracy of your pattern. For example: pattern `(?=.*\bID=(\d+))(?=.*\bUserID=(\d+))` captures `1` and `2` from the string `ID=1 UserID=2`, whereas pattern without `\b`: `(?=.*ID=(\d+))(?=.*UserID=(\d+))` returns the wrong groups: `2` and `2`.
4. In Hive pattern for the external table in SERDEPROPERTIES `input.regex` should describe the whole input string, add `.*` at the end of the pattern.
5. Don't forget that for the beginning of string should also be covered. That's why use the pattern `.*?` for lazy initialization of future patterns.

To sum up, you can create your first regex for parsing Id from posts!

<b>Question!</b> What will be your first regex for parsing Id from the posts? Don't forget to add steps 4 and 5!

<div class="panel-group">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" href="#collapse-answer">Check your answer!</a>
      </h4>
    </div>
    <div id="collapse-answer" class="panel-collapse collapse">
      <div class="panel-body">The correct answer is `".*?(?=.*\\bId=\"(\\d+)\").*"` </div>
    </div>
  </div>
</div>

Let's create the first external table with one row which contains only `Id` field. Let's name it `posts_external_only_id`.
You can watch the lecture for the SerDe format: <a href="https://www.coursera.org/learn/big-data-analysis/lecture/wAGe6/hive-analytics-regexserde-views">Serde Format</a> and <a href="/notebooks/demos/course02_week02-Demo_submission.ipynb#2.-Creation-the-external-table">creation of external table</a> tutorial.

In [6]:
%%writefile demo_example.hql

-- adding necessary JARs and including database
ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;
ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar;

USE demodb;
DROP TABLE IF EXISTS posts_external_only_id;


-- Create external table 
Create EXTERNAL TABLE posts_external_only_id (
                id STRING
)
-- Your code here
ROW FORMAT
    SERDE 'org.apache.hadoop.hive.contrib.serde2.RegexSerDe'
    WITH SERDEPROPERTIES (
    "input.regex" = '.*?(?=.*\\bId=\"(\\d+)\").*'
)
LOCATION '/data/stackexchange1000/posts';

Writing demo_example.hql


In [7]:
!hive -f demo_example.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar]
OK
Time taken: 1.053 seconds
OK
Time taken: 0.141 seconds
OK
Time taken: 0.863 seconds


Hooray! You have created your first table. Let us watch for this table!

In [8]:
%%writefile describe.hql

ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;

USE demodb;
DESCRIBE posts_external_only_id;

Writing describe.hql


In [9]:
!hive -f describe.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
OK
Time taken: 1.072 seconds
OK
id                  	string              	from deserializer   
Time taken: 1.208 seconds, Fetched: 1 row(s)


Let's see that the data is  correctly parsed. For this case, take a select query that chooses for us first 10 rows

In [10]:
%%writefile my_first_select.hql
-- Your code here
USE demodb;
SELECT * FROM posts_external_only_id LIMIT 10;

Writing my_first_select.hql


In [11]:
!hive -f my_first_select.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.16 seconds
OK
NULL
1394
3543
4521
8689
9062
14671
16307
18780
18929
Time taken: 1.688 seconds, Fetched: 10 row(s)


How many posts are there in the dataset? Don't forget to clear the `NULL` values!

In [12]:
%%writefile how_many_posts.hql
-- Your code here

ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;

USE demodb;
SELECT COUNT(*)
FROM posts_external_only_id
WHERE id IS NOT NULL;

Writing how_many_posts.hql


In [13]:
!hive -f how_many_posts.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
OK
Time taken: 1.081 seconds
Query ID = jovyan_20181113111818_0128f04b-c962-4128-9da2-2c7e50dc4452
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks determined at compile time: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1542041219823_0003, Tracking URL = http://c5bd5fec03a1:8088/proxy/application_1542041219823_0003/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1542041219823_0003
Hadoop job information for Stage-1: number of

Try to parse different fields: for example, day and month of creation date. Don't forget that Hive will accept values for the capturing group in the lookahead. 

Now you are ready to complete your task! Before this you can check your regular expression for parsing!

In [14]:
import re

In [15]:
CHECK_ROW = '<row Id="1394" PostTypeId="2" ParentId="1390" CreationDate="2008-08-04T16:38:03.667" Score="16" Body="&lt;p&gt;Not sure how credible &lt;a href=&quot;http://www.builderau.com.au/program/windows/soa/Getting-started-with-Windows-Server-2008-Core-edition/0,339024644,339288700,00.htm&quot; rel=&quot;nofollow noreferrer&quot;&gt;this source is&lt;/a&gt;, but:&lt;/p&gt;&#xA;&#xA;&lt;blockquote&gt;&#xA;  &lt;p&gt;The Windows Server 2008 Core edition can:&lt;/p&gt;&#xA;  &#xA;  &lt;ul&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run the file server role.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run the Hyper-V virtualization server role.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run the Directory Services role.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run the DHCP server role.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run the IIS Web server role.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run the DNS server role.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run Active Directory Lightweight Directory Services.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run the print server role.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;/ul&gt;&#xA;  &#xA;  &lt;p&gt;The Windows Server 2008 Core edition cannot:&lt;/p&gt;&#xA;  &#xA;  &lt;ul&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run a SQL Server.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run an Exchange Server.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run Internet Explorer.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run Windows Explorer.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Host a remote desktop session.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run MMC snap-in consoles locally.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;/ul&gt;&#xA;&lt;/blockquote&gt;&#xA;" OwnerUserId="91" LastEditorUserId="1" LastEditorDisplayName="Jeff Atwood" LastEditDate="2008-08-27T13:02:50.273" LastActivityDate="2008-08-27T13:02:50.273" CommentCount="1" />'

In [20]:
CHECK_REGEX = '^<row.*?(?=.*\\bId=\"(\\d+)\").*(?<=\\bCreationDate\\b=\")(\\d*)-(\\d*).*'

In [21]:
result = re.match(CHECK_REGEX, CHECK_ROW)

In [22]:
# Sanity check
assert result.group(0) == CHECK_ROW

In [23]:
# Check that your groups are correct
print(result.groups())

('1394', '2008', '08')


## Step 4. Complete the assignment

In [24]:
%%writefile task1_create_external_table.hql
-- Create external table posts_sample_external with suitable values
-- Your code here

ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;
ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar;

USE demodb;
DROP TABLE IF EXISTS posts_sample_external;

Create EXTERNAL TABLE posts_sample_external (
                id STRING,
                year STRING,
                month STRING
)

ROW FORMAT
    SERDE 'org.apache.hadoop.hive.contrib.serde2.RegexSerDe'
    WITH SERDEPROPERTIES (
    "input.regex" = '^<row.*?(?=.*\\bId=\"(\\d+)\").*(?<=\\bCreationDate\\b=\")(\\d*)-(\\d*).*'
)
LOCATION '/data/stackexchange1000/posts'
TBLPROPERTIES ("skip.header.line.count"="1");

Writing task1_create_external_table.hql


In [25]:
!hive -f task1_create_external_table.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar]
OK
Time taken: 1.078 seconds
OK
Time taken: 0.143 seconds
OK
Time taken: 0.856 seconds


Make sure that you have created your table correctly. Select the first 10 posts from the dataset.

In [26]:
%%writefile task1_check_select.hql

-- Write select query for the first 10 rows
-- Your code here

USE demodb;
SELECT *
FROM posts_sample_external
LIMIT 10;

Writing task1_check_select.hql


In [27]:
!hive -f task1_check_select.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.15 seconds
OK
NULL	NULL	NULL
1394	2008	08
3543	2008	08
4521	2008	08
8689	2008	08
9062	2008	08
14671	2008	08
16307	2008	08
18780	2008	08
18929	2008	08
Time taken: 1.631 seconds, Fetched: 10 row(s)


Create managed table `posts_sample`. Create the partition by the month and by the year. 

In [28]:
%%writefile task1_create_managed_table.hql
-- create managed table
-- Check that this table contains info about year and month
-- Your code here

ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;

USE demodb;
DROP TABLE IF EXISTS posts_sample;

CREATE TABLE posts_sample (
             id STRING
) 
PARTITIONED BY (year STRING, month STRING);

Writing task1_create_managed_table.hql


In [30]:
!hive -f task1_create_managed_table.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
OK
Time taken: 1.047 seconds
OK
Time taken: 0.121 seconds
OK
Time taken: 1.016 seconds


In [34]:
%%writefile task1_create_managed_table_check.hql
-- Check that this table contains info about year and month

ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;

USE demodb;
DESCRIBE posts_sample

Overwriting task1_create_managed_table_check.hql


In [35]:
!hive -f task1_create_managed_table_check.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
OK
Time taken: 1.132 seconds
OK
id                  	string              	                    
year                	string              	                    
month               	string              	                    
	 	 
# Partition Information	 	 
# col_name            	data_type           	comment             
	 	 
year                	string              	                    
month               	string              	                    
Time taken: 0.997 seconds, Fetched: 9 row(s)


Populate data from the table `posts_sample_external` to the table `posts_sample`. Don't forget about the partitioning rules!

In [50]:
%%writefile task1_insert_table.hql

-- Insert data to the managed table

set hive.exec.max.dynamic.partitions=300;
set hive.exec.max.dynamic.partitions.pernode=256;

ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;

USE demodb;
-- filling managed posts table from external one
SET hive.exec.dynamic.partition.mode=nonstrict;

-- Your code here for inserting data
FROM posts_sample_external
INSERT OVERWRITE TABLE posts_sample
PARTITION (year, month)
SELECT id, year, month
WHERE year IS NOT NULL and month is not NULL;

Overwriting task1_insert_table.hql


In [51]:
!hive -f task1_insert_table.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
OK
Time taken: 1.085 seconds
Query ID = jovyan_20181113114646_3c75ad16-53c1-414b-82d8-94ca468167b2
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1542041219823_0005, Tracking URL = http://c5bd5fec03a1:8088/proxy/application_1542041219823_0005/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1542041219823_0005
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2018-11-13 11:46:53,991 Stage-1 map = 0%,  reduce = 0%
2018-11-13 11:47:12,440 Stage-1 map = 57%,  reduce = 0%, Cumulative CPU 18.46 sec
2018-11-13 11:47:18,867 Stage-1 map = 96%,  reduce = 0%, Cumulative CPU 26.65 sec
2018-11-13 11:47:

Partition demodb.posts_sample{year=2010, month=09} stats: [numFiles=1, numRows=208, totalSize=1664, rawDataSize=1456]
Partition demodb.posts_sample{year=2010, month=10} stats: [numFiles=1, numRows=222, totalSize=1776, rawDataSize=1554]
Partition demodb.posts_sample{year=2010, month=11} stats: [numFiles=1, numRows=236, totalSize=1888, rawDataSize=1652]
Partition demodb.posts_sample{year=2010, month=12} stats: [numFiles=1, numRows=237, totalSize=1896, rawDataSize=1659]
Partition demodb.posts_sample{year=2011, month=01} stats: [numFiles=1, numRows=276, totalSize=2208, rawDataSize=1932]
Partition demodb.posts_sample{year=2011, month=02} stats: [numFiles=1, numRows=273, totalSize=2184, rawDataSize=1911]
Partition demodb.posts_sample{year=2011, month=03} stats: [numFiles=1, numRows=349, totalSize=2792, rawDataSize=2443]
Partition demodb.posts_sample{year=2011, month=04} stats: [numFiles=1, numRows=321, totalSize=2568, rawDataSize=2247]
Partition demodb.posts_sample{year=2011, month=05} stats

MapReduce Jobs Launched: 
Stage-Stage-1: Map: 1   Cumulative CPU: 28.96 sec   HDFS Read: 60003502 HDFS Write: 347300 SUCCESS
Total MapReduce CPU Time Spent: 28 seconds 960 msec
OK
Time taken: 72.443 seconds


Make sure that your table contains appropriate data about posts

In [56]:
%%writefile task1_watch_new_table.hql
-- Your code here
USE demodb;
SELECT * FROM posts_sample LIMIT 10;

Overwriting task1_watch_new_table.hql


In [57]:
!hive -f task1_watch_new_table.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.188 seconds
OK
1394	2008	08
3543	2008	08
4521	2008	08
8689	2008	08
9062	2008	08
14671	2008	08
16307	2008	08
18780	2008	08
18929	2008	08
19668	2008	08
Time taken: 2.061 seconds, Fetched: 10 row(s)


Take the third row of the dataset in the ascending order for the posts (firstly by year, after that by month)

In [83]:
%%writefile task1_result.hql
-- Your code here

ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;

USE demodb;

SELECT concat_ws("\t", year, concat_ws("-", year, month), c)
FROM (
    SELECT year, month, STRING(COUNT(*)) as c, ROW_NUMBER() OVER (ORDER BY year, month) as row
    FROM posts_sample
    GROUP BY year, month
) as tmp
WHERE row=3;

Overwriting task1_result.hql


In [84]:
!hive -f task1_result.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
OK
Time taken: 1.077 seconds
Query ID = jovyan_20181113121313_19b1a443-13e3-4838-aad4-17b0165af750
Total jobs = 2
Launching Job 1 out of 2
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1542041219823_0009, Tracking URL = http://c5bd5fec03a1:8088/proxy/application_1542041219823_0009/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1542041219823_0009
Hadoop job information for

## Step 5. Submission part. Do not touch!! And simple run all cells below!

Copy your notebook from the steps <a href="#Step-4.-Complete-the-assignment">Step 4</a> and <a href="#Step-5.-Submission-part.-Do-not-touch!!-And-simple-run-all-cells-below!">Step 5</a> to the new notebook. Run all the cells! And submit the copied notebook!

In [ ]:
!cat task1_create_external_table.hql > task1.hql
!cat task1_create_managed_table.hql >> task1.hql
!cat task1_insert_table.hql >> task1.hql
!cat task1_result.hql >> task1.hql

Take a look at your submission query!

In [ ]:
!cat task1.hql

In [ ]:
%%javascript

$(document).ready(function() {
    console.log('Ready');
    
    
    function is_hive_command(list_tokens) {
        return list_tokens.indexOf('hive') > -1 && 
             list_tokens.indexOf('f') > -1 &&
             list_tokens.indexOf('-') > -1 && 
             list_tokens.indexOf('!') > -1 &&
             list_tokens.indexOf('hql') > -1 && 
             list_tokens.indexOf('writefile') == -1;
    } 
    
    function collectText(input_tag) {

        var result_string = [];
        $.each($(input_tag).children(), function(index, child) {
            result_string.push($(child).text());
        });
        return [result_string, is_hive_command(result_string)];
    };
    
    var filtered_results = $(".cell.code_cell.rendered").filter(function(index, element) {
        var out = collectText($(element).find('.CodeMirror-line').find('span'));
        console.log(out);
        return collectText($(element).find('.CodeMirror-line').find('span'))[1];
    });
    $(filtered_results).remove();
});

In [ ]:
%%bash
hive -f task1.hql

Congratulations! You have completed the assignment! Now you can submit it to the system and get your results!

Copy your notebook from the steps <a href="#Step-4.-Complete-the-assignment">Step 4</a> and <a href="#Step-5.-Submission-part.-Do-not-touch!!-And-simple-run-all-cells-below!">Step 5</a> to the new notebook. Run all the cells! And submit the copied notebook!